In [8]:
%cd /content/driver/My Drive/Github/GraduationThesis
%ls

/content/driver/My Drive/Github/GraduationThesis
bug_report/       GraduationThesis.ipynb  parse_ast.py      source_file/
data_test/        javalang/               README.md
Generate_data.py  pair_bugrp_source.py    requirements.txt


In [ ]:
# kết nối với google drive

from google.colab import drive
drive.mount('/content/driver')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/driver
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
# import thư viện

from glob import glob
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
from nltk import sent_tokenize,word_tokenize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU, Bidirectional
from keras.layers import Dropout
from tensorflow import keras
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# txt => csv
path_train = '/content/driver/My Drive/GraduationThesis/Dataset/dataset/*.txt'
files = glob(path_train)
x_input =[]
y_output=[]
print(len(files))
for file in files:
  df = pd.read_csv(file, sep="\t")
  print(re.sub(".txt", ".csv", file))
  df.to_csv(re.sub(".txt", ".csv", file), index=False) 



5
/content/driver/My Drive/GraduationThesis/Dataset/dataset/Tomcat.csv
/content/driver/My Drive/GraduationThesis/Dataset/dataset/JDT.csv
/content/driver/My Drive/GraduationThesis/Dataset/dataset/SWT.csv
/content/driver/My Drive/GraduationThesis/Dataset/dataset/AspectJ.csv
/content/driver/My Drive/GraduationThesis/Dataset/dataset/Eclipse_Platform_UI.csv


In [ ]:
# đọc file csv
data = pd.read_csv('/content/driver/My Drive/GraduationThesis/Dataset/dataset/AspectJ.csv')
print("data")
print(data)

data
      id  ...                                        Unnamed: 10
0      1  ...  52:tests/src/org/aspectj/systemtest/incrementa...
1      2  ...  63:org.aspectj.matcher/src/org/aspectj/weaver/...
2      3  ...  6:org.aspectj.ajdt.core/src/org/aspectj/ajdt/i...
3      4  ...  5:loadtime/src/org/aspectj/weaver/loadtime/Aj....
4      5  ...  21:weaver/src/org/aspectj/weaver/bcel/BcelClas...
..   ...  ...                                                ...
588  589  ...                                                NaN
589  590  ...                                                NaN
590  591  ...                                                NaN
591  592  ...                                                NaN
592  593  ...                                                NaN

[593 rows x 11 columns]


In [ ]:
# tiền xử lí dữ liệu bugs reports
def processing(row):
    ps = PorterStemmer()
    y_save = []
    x = sent_tokenize(row)
    line_after = ''
    for y1 in x:
        y1 = re.sub(r'\W', ' ', y1)  # loại bỏ kí tự đặc biệt 
        y1 = re.sub(r'\d', ' ', y1)  # loại bỏ số
        words = word_tokenize(y1)
        words=[w for w in words if w not in STOPWORDS]
        words=[w.lower() for w in words]
        for w1 in words:
            if (len(w1) != 1):  # nếu w1 là 1 từ có 1 kí tự thì loại bỏ
                w1 = ps.stem(w1)
                line_after += ' '
                line_after += w1

    y_save.append(line_after.lower())
    return y_save

In [ ]:
# Tiền xử lí
def get_data(data):
    data_summary=data["summary"]
    data_description=data["description"]
    print(data_description)
    data_description=np.array(data_description)
    files=[]
    for summary, desc in zip(data_summary, data_description):
        if not desc or pd.isnull(desc):
            continue
        try:
            summary=processing(summary)
            print(summary)
            summary=''.join(summary)
            print(summary)
            desc=processing(desc)
            print(desc)
            desc=''.join(desc)
            print(desc)
        except:
            continue

In [ ]:
data = pd.read_csv('/content/driver/My Drive/GraduationThesis/Dataset/dataset/AspectJ.csv')
print("data")
print(get_data(data))

data
0                                                    NaN
1      It would be good to be able to insert the encl...
2      The -Xlintfile option is not a great fit for c...
3      When I enable JMX remote management on a JVM a...
4                                                    NaN
                             ...                        
588    The org.aspectj.ajde.BuildOptionsAdapter inter...
589    I use aspectj1.1b4 (the version list box did n...
590                                                  NaN
591    If you don't find the exception below in a bug...
592    The aspect below fails to compile with 1.1b2, ...
Name: description, Length: 593, dtype: object
[' it good abl insert enclos class enclos member joinpoint']
 it good abl insert enclos class enclos member joinpoint
[' the xlintfil option great fit control messag multipl build project specif case pluginmanag section maven parent pom the problem need local file configur messag output level ignor warn error want specif

In [ ]:
# Mẫu mô hình LSTM đã làm
n_input = 3
n_features = 2

model = Sequential()
## K59
model.add(LSTM(20, activation='relu', input_shape=(n_input, n_features)))


## 3 tầng lstm
# model.add(LSTM(20, return_sequences=True, activation='relu', input_shape=(n_input, n_features)))
# model.add(LSTM(20, return_sequences=True))
# model.add(LSTM(20))

## GRU
# model.add(GRU(20, activation='relu', input_shape=(n_input, n_features)))

## BiLSTM
# model.add(Bidirectional(LSTM(20, activation='relu', return_sequences=True),input_shape=(n_input, n_features)))
# model.add(Bidirectional(LSTM(20, activation='relu')))

# model.add(Dropout(0.3))
model.add(Dropout(0.2))
model.add(Dense(1))
model.summary()
adam = Adam(lr=0.001)
model.compile(optimizer=adam, loss='mse')